# Fractional Cascading

## Problem Statement

Given a `k` list of sorted integers and a value `x`. Given a query value `x` return the largest value less than or equal to `x` in each of the `k` lists.

In [1]:
import bisect

In [2]:
arr = [
    [24, 64, 65, 80, 93],
    [23, 25, 26],
    [13, 44, 62, 66],
    [11, 35, 46, 79, 81],
]

## Slower Queries Less Space | Naive Way - `k` binary searches

In [3]:
def get_locations_1(x): 
    return [bisect.bisect_left(l, x) for l in arr]

In [4]:
get_locations_1(50)

[1, 3, 2, 3]

## Faster Queries More Space | Global Ordering

In [5]:
# For each of the element in `arr` positions will hold
# the location where the element `arr[i][j]` will be inserted.
positions = []
for i in range(len(arr)):
    positions.append([])
    for j in range(len(arr[i])):
        positions[i].append([[]] * len(arr[i]))
        positions[i][j] = [-1] * len(arr)

In [6]:
for i, l in enumerate(arr):
    for j, e in enumerate(l):
        for k, m in enumerate(arr):
            positions[i][j][k] = int(bisect.bisect_left(m, e))

See where `17` - 4th list 4th element

In [7]:
positions[3][3]

[3, 3, 4, 3]

In [8]:
pos_arr = sorted([
    (y, positions[i][j],)
    for i, x in enumerate(arr)
        for j, y in enumerate(x)
], key=lambda x: x[0])

In [9]:
pos_arr[:5]

[(11, [0, 0, 0, 0]),
 (13, [0, 0, 0, 1]),
 (23, [0, 0, 1, 1]),
 (24, [0, 1, 1, 1]),
 (25, [1, 1, 1, 1])]

In [10]:
U = list(zip(*pos_arr))[0]

In [11]:
def get_locations_2(x): 
    return list(pos_arr[bisect.bisect_left(U, x)][1])

In [12]:
get_locations_2(50)

[1, 3, 2, 3]

## Best of both Worlds | Fractional Cascading

In [13]:
m_arr = []

In [14]:
m_arr.insert(0, [x for x in arr[-1]])

In [15]:
for i in range(len(arr) - 2, -1, -1):
    m_arr.insert(0, sorted([x for k, x in enumerate(m_arr[0]) if k % 2] + arr[i]))

In [16]:
m_arr

[[24, 25, 35, 64, 65, 79, 80, 93],
 [23, 25, 26, 35, 62, 79],
 [13, 35, 44, 62, 66, 79],
 [11, 35, 46, 79, 81]]

In [17]:
# For each of the element in `arr` positions will hold
# the location where the element `arr[i][j]` will be inserted.
pointers = []
for i in range(len(m_arr)):
    pointers.append([])
    for j in range(len(m_arr[i])):
        pointers[i].append([[]] * len(arr[i]))
        pointers[i][j] = [-1] * 2

In [18]:
for i, l in enumerate(m_arr):
    for j, m in enumerate(m_arr[i]):
        pointers[i][j][0] = bisect.bisect_left(arr[i], m_arr[i][j])
        if i == len(m_arr) - 1:
            pointers[i][j][1] = 0
        else:
            pointers[i][j][1] = bisect.bisect_left(m_arr[i+1], m_arr[i][j])

In [19]:
pointers

[[[0, 1], [1, 1], [1, 3], [1, 5], [2, 5], [3, 5], [3, 6], [4, 6]],
 [[0, 1], [1, 1], [2, 1], [3, 1], [3, 3], [3, 5]],
 [[0, 1], [1, 1], [1, 2], [2, 3], [3, 3], [4, 3]],
 [[0, 0], [1, 0], [2, 0], [3, 0], [4, 0]]]

In [20]:
def get_locations_3(x): 
    locations = []
    loc, next_loc = pointers[0][bisect.bisect_left(m_arr[0], x)]
    locations.append(loc)
    for i in range(1, len(m_arr)):
        if next_loc == 0:
            loc, next_loc = 0, 0
        else:    
            if x < m_arr[i][next_loc-1]:
                loc, next_loc = pointers[i][next_loc-1]
            else:
                loc, next_loc = pointers[i][next_loc]
        locations.append(loc)
    return locations

In [21]:
get_locations_3(50)

[1, 3, 2, 3]